# DeepMatter Data Science/Cheminformatician Interview Notebook

This Jupyter notebook contains tasks to be completed prior to your second interview, and is expected to take
2-3 hours. Please complete it as necessary and be prepared to discuss it at the interview. Code should be
completed using Python, unless a viable/better/justifiable alternative exists.

## Environmental Setup

Projects in DeepMatter are structured using Anaconda and Pipenv. As these questions will
make use of RDKit, Anaconda is the preferred virtual environment setup. Before you continue,
please install Anaconda and create an environment named "dm_interview".

## Task 1 - Runtime Sensor Data

Clone the `dm_datascience` repository located on Github at
https://github.com/deepmatterltd/dm_datascience, and unzip the data located under
`02_PCML_and_PCRR/data/pcrr.zip`.

PCRR (Practical Chemistry Runtime Record) files are an XML-based method of capturing all data
associated with a chemistry run in our product DigitalGlassware. They capture things such as the
operations associated with the reaction, the reagents, the product, the final yield and other outcomes.
They also contain observations such as textual and photo notes, as well as timestamps associated
with when each operation was performed (e.g. *add 5mg of catalyst X to reactor vessel*).

0. Download and Extract Data

In [3]:
import zipfile
import os

# clone repo
dm_datascience_repo_dir = "../dm_datascience"
if not os.path.isdir(dm_datascience_repo_dir): # only clone if the directory doesn't exist already
    !git clone https://github.com/deepmatterltd/dm_datascience $dm_datascience_repo_dir


zip_file_path = os.path.join(dm_datascience_repo_dir, "02_PCML_and_PCRR", "data", "pcrr.zip") # location of target zip file
pccr_dir = os.path.join("..", "pcrr") # target extraction directory

# extract zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(pccr_dir)

1. For each of the PCRR files in the run, extract the name of the recipe, author and number of
operations in the recipe.

In [10]:
import pathlib
import xml.dom.pulldom as pulldom
import xml.etree.ElementTree as ET
import json

pccr_file_ext = ".pcrr"
pccr_files = os.listdir(pccr_dir) # get all files in the directory

recipe_information = []
for pccr_file in pccr_files: # iterate over each file

    if pathlib.Path(pccr_file).suffix != pccr_file_ext:
        print(f"{pccr_file} is not a .pccr file, skipping.")
        continue
    
    pccr_file_path = os.path.join(pccr_dir,pccr_file) # get the full path

    doc = pulldom.parse(pccr_file_path) # these xml files are very large, use pulldom to extract the parts we need 
    for event, node in doc:
        if event == pulldom.START_ELEMENT and node.tagName == "recipe_version": # all required information is within this tag
            doc.expandNode(node) # expand the node so we can parse it with elementree and xpath
            root = ET.fromstring(node.toxml())  # load the xml into elementree

            # use xpath to find recipe name and author
            name = root.find("./recipe").attrib["name"]
            author = root.find("./pcml/meta/author").text

            # use xpath to find all operation tags, and then count them
            operations = len(root.findall("./pcml/step/group/operation"))
            recipe_information.append({"filename":pccr_file, "name": name, "author": author, "operations": operations})

            break # we don't need any more information, stop processing the file
            
print(json.dumps(recipe_information, indent=4))

[
    {
        "filename": "3a_run_02.pcrr",
        "name": "3a) Synthesis of N-(1-Naphthoyl)-4-methylbenzenesulfonohydrazide",
        "author": "Deepmatter",
        "operations": 84
    },
    {
        "filename": "3a_run_03.pcrr",
        "name": "3a) Synthesis of N-(1-Naphthoyl)-4-methylbenzenesulfonohydrazide",
        "author": "Deepmatter",
        "operations": 84
    },
    {
        "filename": "3a_run_01.pcrr",
        "name": "3a) Synthesis of N-(1-Naphthoyl)-4-methylbenzenesulfonohydrazide",
        "author": "Deepmatter",
        "operations": 84
    }
]


2. XML is great for explicitly structuring data, but not much fun to handle for signal processing.
Write a function which can take in a PCRR filename and sensor name (e.g. `irObjTempI` for immersed temperature, or
`uvaI` for immersed UVA level) and returns the data a Pandas Series of length N. Note that each sensor
reading in the XML has a timestamp associated with it. This timestamp should also be parsed and assigned
to the index of the Series object returned.

In [50]:
import pandas as pd

def extract_sensor_data(pcrr_file_path:str,
                        sensor_name:str,
                        stop_after_first:bool=False):
    """Extracts values and timestamps from the given pcrr file and sensor.
    pccr_file_path: Path to the pcrr file
    sensor_name: The name of the sensor to read data for
    stop_after_first: Defaults to False. Set True to stop parsing the document after the first instance of the sensor is found.
    """
    
    pccr_file_ext = ".pcrr"
    if pathlib.Path(pccr_file).suffix != pccr_file_ext:
        print(f"{pccr_file} is not a .pccr file, aborting.")
        return None

    sensor_data_value = []
    sensor_data_timestamp = []

    doc = pulldom.parse(pcrr_file_path) # these xml files are very large, use pulldom to extract the parts we need
    for event, node in doc:
        if event == pulldom.START_ELEMENT:
            if node.tagName == "sensor_data" and node.getAttribute("name") == sensor_name:
                doc.expandNode(node)
                root = ET.fromstring(node.toxml())

                sensor_data_value += [x.text for x in root.findall("./sensor_data_record/value")]
                sensor_data_timestamp += [x.text for x in root.findall("./sensor_data_record/timestamp")]

                if stop_after_first:
                    break

    df = pd.DataFrame(data={sensor_name: sensor_data_value, "timestamp": sensor_data_timestamp})
    df = df.astype({sensor_name: float, "timestamp": "datetime64[ns]"})
    df.set_index("timestamp", inplace=True)
    
    return df

3. Use the function to extract immersed temperature (`irObjTempI`) and plot against time.

In [56]:
file_path = "../pcrr/3a_run_01.pcrr"
sensor_name = "irObjTempI"
stop_after_first = True

df_sensor_data = extract_sensor_data(file_path, sensor_name, stop_after_first)

<AxesSubplot:xlabel='timestamp'>

4. Compute the rolling median of the immersed temperature, with a window of 30 seconds, and plot it.
Overlay the rolling median of the immersed UVA trace (`uvaI`) on the same plot.

## Task 2 - Reaction Data

While PCRR is used as a means of encapsulating chemistry runs in DigitalGlassware, the concept
 of using XML as a structuring mechanism is quite recent. There are many older datasets which
 use formats such as RDF, wherein lists of reactions describe the structural change which
 occurs during the chemistry. This section will focus on a small sample dataset which uses
 this format

To perform this task, install [RDKit](https://www.rdkit.org) to your Anaconda environment.

An RD file has been provided under the `data` directory for use in the following tasks.

1. Write a parser to split up `data/spresi-100.rdf` into `$RXN` blocks, which
denote individual reactions. A valid reaction starts with `$RXN` and includes
every subsequent line until the next `$RXN` block (inclusive) or the
end of the file.
Everything before the first `$RXN` block can be ignored.
<p>
Alternatively, there is a Python library which will do this for you, but it's up to you
to find it.


2. Use RDKit to parse each of the `$RXN` text blocks you parsed in above,
 and print out the SMILES for the reagents and products
 on the first 5 reactions.


2. Generate [molecular fingerprints](https://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints)
for all reagents and products and store them in a binary numpy matrix (bonus: use a sparse matrix).
Use a radius of 2 and fingerprint length of 1000. Ignore any molecules which throw an exception, and remove them from
the final array.

3. Perform dimensionality reduction on the fingerprints to visualise the data in 2D.
Use whether the molecule's fingerprint was a reactant or product as the colour of the point.

Perform clustering on the fingerprints and visualise the results in the same embedding used
in the previous question.

## Wrap Up

Please bring the completed code in this notebook/code for generating results along to your interview.
The code should be executed in real-time unless there is a good reason to avoid this. You will be
asked how you completed the tasks and why you did it a certain way, as well as discussion
on other ways the tasks could have been performed.

